In [146]:
from sympy import *
import regex

pax, pay, pbx, pby, vax, vay, vbx, vby, aax, aay, abx, aby, pix, piy = symbols('p_a_x p_a_y p_b_x p_b_y v_a_x v_a_y v_b_x v_b_y a_a_x a_a_y a_b_x a_b_y p_i_x p_i_y',real=True)
F, T = symbols('F T',real=True,positive=True)


def gdcode(expr):
    expr = collect_const((expr))
    rhs = ccode(expr)
    opc = count_ops(expr)
    # turn squares into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*2\s*\)', r'(\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-2\s*\)', r'(1.0/(\1*\1))', rhs)
    # turn cubes into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*3\s*\)', r'(\1*\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-3\s*\)', r'(1.0/(\1*\1*\1))', rhs)
    # evaluate fractions
    #rhs = regex.sub(r'(\d+\.?\d*)/(\d+\.?\d*)', lambda match: str(float(match.group(1))/float(match.group(2))), rhs)
    # convert integers to floats
    rhs = regex.sub(r'(?<!\.)\b(\d+)\b(?!\.)',r'\1.0',rhs)
    # rename vector parts
    rhs = regex.sub(r'\b(\w+)_(x|y)',r'\1.\2',rhs)
    # fix ternary if
    rhs = regex.sub(r'\(\s*\(\s*(.*?)\s*\)\s*\?\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\n\s*:\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\)', r'\2 if \1 else \3', rhs)
    return (rhs,opc)

def make_code(qs):
    cse_qs = cse(qs,symbols=numbered_symbols('_'))
    qs = cse_qs[0] + [(q.lhs,q.rhs) for q in cse_qs[1]]
    code=''
    opcount=0
    for q in qs:
        rhs, opc = gdcode(q[1])
        opcount+=opc
        code+='var '+str(q[0])+':float = '+rhs
        code+='\n'
    
    code = '# opcount: '+str(opcount)+'\n'+code
    code = '# generated by ballistics_gen.ipynb\n' + code

    return code


def solve_quart(a,b,c,d,e):
    p1 = 2*c**3 - 9*b*c*d + 27*a*d**2 + 27*e*b**2 - 72*a*c*e
    p2 = p1 + sqrt(-4*(c**2 - 3*b*d + 12*a*e)**3 + p1**2)
    p3 = (c**2 - 3*b*d + 12*a*e)/(3*a*cbrt(p2/2)) + cbrt(p2/2)/(3*a)
    p4 = sqrt(b**2/(4*a**2) - 2*c/(3*a) + p3)
    p5 = b**2/(2*a**2) - 4*c/(3*a) - p3
    p6 = (-b**3/(a*3) + 4*b*c/(a**2) - 8*d/a)/(4*p4)
    return [
        -b/(4*a) - p4/2 - sqrt(p5-p6)/2,
        -b/(4*a) - p4/2 + sqrt(p5-p6)/2,
        -b/(4*a) + p4/2 - sqrt(p5+p6)/2,
        -b/(4*a) + p4/2 + sqrt(p5+p6)/2
    ]

In [147]:
# linear-linear intercept
q_pix = Eq(vax*T, vbx*T + pbx)
q_piy = Eq(vay*T, vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
#q_ca = Eq(ca, q_f2_terms[T**4])
#q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [Eq(T,q.subs({q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs})) for q in q_t]
print(make_code(sols))
print(make_code([q_vax,q_vay]))

Eq(T*v_a_x, T*v_b_x + p_b_x)

Eq(T*v_a_y, T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, v_b_x + p_b_x/T)

Eq(v_a_y, v_b_y + p_b_y/T)

...

Eq(c_d, -8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_b_x**2 - 4*p_b_y**2)

T**2*c_c + T*c_d + c_e

[(-c_d - sqrt(-4*c_c*c_e + c_d**2))/(2*c_c),
 (-c_d + sqrt(-4*c_c*c_e + c_d**2))/(2*c_c)]

# generated by ballistics_gen.ipynb
# opcount: 28
var _0:float = 8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var _1:float = 4.0*((F*F) - (v_b.x*v_b.x) - (v_b.y*v_b.y))
var _2:float = sqrt((_0*_0) - 4.0*_1*(-4.0*(p_b.x*p_b.x) - 4.0*(p_b.y*p_b.y)))
var _3:float = (1.0/2.0)/_1
var T:float = _3*(_0 - _2)
var T:float = _3*(_0 + _2)

# generated by ballistics_gen.ipynb
# opcount: 5
var _0:float = 1.0/T
var v_a_x:float = _0*p_b.x + v_b.x
var v_a_y:float = _0*p_b.y + v_b.y



In [157]:
# linear-quadratic intercept
q_pix = Eq(vax*T, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(vay*T, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cb, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cb.rhs:q_cb.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

#q_t = solve(q_f2, T)
q_t = solve_quart(ca,cb,cc,cd,ce)
display(q_t)

sols = [Eq(T,q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs})) for q in q_t]
#print(make_code([q_ca,q_cb,q_cc,q_cd,q_ce]))
print(make_code(sols))
print(make_code([q_vax,q_vay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))
display(sols[0])

Eq(T*v_a_x, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T*v_a_y, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, T*a_b_x/2 + v_b_x + p_b_x/T)

Eq(v_a_y, T*a_b_y/2 + v_b_y + p_b_y/T)

...

Eq(c_e, -4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**3*c_b + T**2*c_c + T*c_d + c_e

[-sqrt(3)*sqrt(48*2**(1/3)*c_e/(-72*c_a*c_c*c_e + 27*c_a*c_d**2 + 27*c_b**2*c_e - 9*c_b*c_c*c_d + 2*c_c**3 + sqrt(-6912*c_a**3*c_e**3 + 5184*c_a**2*c_b*c_d*c_e**2 + 3456*c_a**2*c_c**2*c_e**2 - 3888*c_a**2*c_c*c_d**2*c_e + 729*c_a**2*c_d**4 - 3888*c_a*c_b**2*c_c*c_e**2 + 162*c_a*c_b**2*c_d**2*c_e + 2160*c_a*c_b*c_c**2*c_d*c_e - 486*c_a*c_b*c_c*c_d**3 - 432*c_a*c_c**4*c_e + 108*c_a*c_c**3*c_d**2 + 729*c_b**4*c_e**2 - 486*c_b**3*c_c*c_d*c_e + 108*c_b**3*c_d**3 + 108*c_b**2*c_c**3*c_e - 27*c_b**2*c_c**2*c_d**2))**(1/3) - 12*2**(1/3)*c_b*c_d/(c_a*(-72*c_a*c_c*c_e + 27*c_a*c_d**2 + 27*c_b**2*c_e - 9*c_b*c_c*c_d + 2*c_c**3 + sqrt(-6912*c_a**3*c_e**3 + 5184*c_a**2*c_b*c_d*c_e**2 + 3456*c_a**2*c_c**2*c_e**2 - 3888*c_a**2*c_c*c_d**2*c_e + 729*c_a**2*c_d**4 - 3888*c_a*c_b**2*c_c*c_e**2 + 162*c_a*c_b**2*c_d**2*c_e + 2160*c_a*c_b*c_c**2*c_d*c_e - 486*c_a*c_b*c_c*c_d**3 - 432*c_a*c_c**4*c_e + 108*c_a*c_c**3*c_d**2 + 729*c_b**4*c_e**2 - 486*c_b**3*c_c*c_d*c_e + 108*c_b**3*c_d**3 + 108*c_b**2*c_c**3*c

# generated by ballistics_gen.ipynb
# opcount: 199
var _0:float = 4.0*a_b.x
var _1:float = 4.0*a_b.y
var _2:float = -(_0*v_b.x + _1*v_b.y)
var _3:float = (_2*_2*_2)
var _4:float = -((a_b.x*a_b.x) + (a_b.y*a_b.y))
var _5:float = 1.0/_4
var _6:float = sqrt(3.0)
var _7:float = (_2*_2)
var _8:float = (_4*_4)
var _9:float = 1.0/_8
var _10:float = 4.0*(F*F) - (_0*p_b.x + _1*p_b.y + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var _11:float = 8.0*_10*_5
var _12:float = -4.0*((p_b.x*p_b.x) + (p_b.y*p_b.y))
var _13:float = cbrt(2.0)
var _14:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var _15:float = (_14*_14)
var _16:float = 27.0*_15
var _17:float = _12*_7
var _18:float = (_10*_10*_10)
var _19:float = _10*_2
var _20:float = _12*_4
var _21:float = (_14*_14*_14)
var _22:float = (_12*_12)
var _23:float = _15*_4
var _24:float = _22*_8
var _25:float = _14*_2
var _26:float = 108.0*_18
var _27:float = (_10*_10)
var _28:float = 3888.0*_10
var _29:float = cbrt(-72.0*_10*_20 - 9.0*_14*_19 + _16*_4 + 27.0*_17 + 2.

Eq(T, -sqrt(3)*sqrt(48*2**(1/3)*(-4*p_b_x**2 - 4*p_b_y**2)/(-72*(-a_b_x**2 - a_b_y**2)*(-4*p_b_x**2 - 4*p_b_y**2)*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2) + 27*(-a_b_x**2 - a_b_y**2)*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y)**2 + 27*(-4*p_b_x**2 - 4*p_b_y**2)*(-4*a_b_x*v_b_x - 4*a_b_y*v_b_y)**2 - 9*(-4*a_b_x*v_b_x - 4*a_b_y*v_b_y)*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y)*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2) + 2*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2)**3 + sqrt(-6912*(-a_b_x**2 - a_b_y**2)**3*(-4*p_b_x**2 - 4*p_b_y**2)**3 + 5184*(-a_b_x**2 - a_b_y**2)**2*(-4*p_b_x**2 - 4*p_b_y**2)**2*(-4*a_b_x*v_b_x - 4*a_b_y*v_b_y)*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y) + 3456*(-a_b_x**2 - a_b_y**2)**2*(-4*p_b_x**2 - 4*p_b_y**2)**2*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2)**2 - 3888*(-a_b_x**2 - a_b_y**2)**2*(-4*p_b_x**2 - 4*p_b_y**2)*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y)**2*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x

In [149]:
# quadratic-linear intercept

q_pix = Eq(aax*(T**2)/2 , vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2 , vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
#q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [Eq(T,q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs})) for q in q_t]
print(make_code(sols))
print(make_code([q_aax,q_aay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))

Eq(T**2*a_a_x/2, T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2, T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, 2*(T*v_b_x + p_b_x)/T**2)

Eq(a_a_y, 2*(T*v_b_y + p_b_y)/T**2)

...

Eq(c_d, -8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**2*c_c + T*c_d + c_e

[Piecewise((-sqrt(-2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 2*c_c/(3*c_a))/2 - sqrt(2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 4*c_c/(3*c_a) + 2*c_d/(c_a*sqrt(-2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 2*c_c/(3*c_a))))/2, Eq(c_e/c_a + c_c**2/(12*c_a**2), 0)), (-sqrt(-2*(-c_e/c_a - c_c**2/(12*c_a**2))/(3*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) + c_c**3/(216*c_a**3))**(1/3)) + 2*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) + c_c**3/(216*c_a**3))**(1/3) - 2*c_c/(3*c_a))/2 - sqrt(2*(-c_e/c_a - c_c**2/(12*c_a**2))/(3*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) 

# generated by ballistics_gen.ipynb
# opcount: 118
var _0:float = (1.0/(F*F))
var _1:float = -4.0*((v_b.x*v_b.x) + (v_b.y*v_b.y))
var _2:float = _0*_1
var _3:float = (2.0/3.0)*_2
var _4:float = pow(F, -4.0)
var _5:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var _6:float = _4*(_5*_5)
var _7:float = (_1*_1*_1)/pow(F, 6.0)
var _8:float = -4.0*((p_b.x*p_b.x) + (p_b.y*p_b.y))
var _9:float = (1.0/3.0)*_1*_4*_8 - 1.0/8.0*_6 - 1.0/108.0*_7
var _10:float = 2.0*cbrt(_9)
var _11:float = sqrt(-_10 - _3)
var _12:float = (1.0/2.0)*_11
var _13:float = (4.0/3.0)*_2
var _14:float = -_13
var _15:float = 2.0*_0*_5
var _16:float = _15/_11
var _17:float = (1.0/2.0)*sqrt(_10 + _14 + _16)
var _18:float = _0*_8 + (1.0/12.0)*(_1*_1)*_4
var _19:float = _18 == 0.0
var _20:float = -_18
var _21:float = cbrt(-1.0/6.0*_1*_4*_8 + (1.0/16.0)*_6 + (1.0/216.0)*_7 + sqrt((1.0/27.0)*(_20*_20*_20) + (1.0/4.0)*(_9*_9)))
var _22:float = 2.0*_21
var _23:float = (2.0/3.0)*_20/_21
var _24:float = -_22 + _23
var _25:float = sqrt(-_

In [150]:
# quadratic-quadratic intercept

q_pix = Eq(aax*(T**2)/2, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cb, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cb.rhs:q_cb.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [Eq(T,q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs})) for q in q_t]
print(make_code(sols))
print(make_code([q_aax,q_aay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))

Eq(T**2*a_a_x/2, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, a_b_x + 2*v_b_x/T + 2*p_b_x/T**2)

Eq(a_a_y, a_b_y + 2*v_b_y/T + 2*p_b_y/T**2)

...

Eq(c_d, -8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**3*c_b + T**2*c_c + T*c_d + c_e

[Piecewise((-sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2))/2 - sqrt((2*c_d/c_a - c_b*c_c/c_a**2 + c_b**3/(4*c_a**3))/sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2)) + 2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 4*c_c/(3*c_a) + c_b**2/(2*c_a**2))/2 - c_b/(4*c_a), Eq(c_e/c_a - c_b*c_d/(4*c_a**2) + c_c**2/(12*c_a**2), 0)), (-sqrt(-2*(-c_e/c_a + c_b*c_d/(4*c_a**2) - c_c**2/(12

# generated by ballistics_gen.ipynb
# opcount: 172
var _0:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var _1:float = (F*F) - ((a_b.x*a_b.x) + (a_b.y*a_b.y))
var _2:float = 1.0/_1
var _3:float = _0*_2
var _4:float = 4.0*a_b.x
var _5:float = 4.0*a_b.y
var _6:float = -(_4*v_b.x + _5*v_b.y)
var _7:float = (1.0/(_1*_1*_1))
var _8:float = (_6*_6*_6)*_7
var _9:float = -(_4*p_b.x + _5*p_b.y + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var _10:float = (1.0/(_1*_1))
var _11:float = _10*_6
var _12:float = _11*_9
var _13:float = -_12 + 2.0*_3 + (1.0/4.0)*_8
var _14:float = (_6*_6)
var _15:float = _2*_9
var _16:float = (2.0/3.0)*_15
var _17:float = _10*_14
var _18:float = -3.0/8.0*_17 + _2*_9
var _19:float = (_18*_18*_18)
var _20:float = pow(-1.0/2.0*_12 + _3 + (1.0/8.0)*_8, 2.0)
var _21:float = -1.0/4.0*_0*_11 + _2*(-4.0*(p_b.x*p_b.x) - 4.0*(p_b.y*p_b.y))
var _22:float = (1.0/16.0)*_14*_7*_9 + _21 - 3.0/256.0*pow(_6, 4.0)/pow(_1, 4.0)
var _23:float = (1.0/3.0)*_18*_22 - 1.0/108.0*_19 - 1.0/8.0*_20
var _2